In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv0 = nn.Sequential(         
            nn.Conv2d(32, 32*8, 1, 1, padding = "same",),
            nn.BatchNorm2d(32*8),
            nn.ReLU(),
            nn.Conv2d(32*8, 32*8, 3, 1, padding = "same",),
            nn.BatchNorm2d(32*8),
            nn.ReLU(),
            nn.Conv2d(32*8, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        identity = x
        x = self.conv0(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv0): Sequential(
    (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(


# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0444
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 2.9171
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.7827
10.701545715332031
epoch: 01 | Accuracy: 10.70 Loss: 79.9416

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.5412
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.3194
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.2958
27.824018478393555
epoch: 02 | Accuracy: 27.82 Loss: 65.2811

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 1.8612
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 1.4487
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.4347
52.91320037841797
epoch: 03 | Accuracy: 52.91 Loss: 45.7147

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 0.7322
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 0.8895
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.1014
72.88941955566406
epoch: 04 | Accuracy: 72.89 Loss: 27.1942

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 0.5586
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.5736
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 0.6428
84.06658935546875
epoch: 05 | Accuracy: 84.07 Loss: 16.1411

Epoch: [6/30]	|	Step: [

In [7]:
print(Accy_list)

[10.701545715332031, 27.824018478393555, 52.91320037841797, 72.88941955566406, 84.06658935546875, 93.10344696044922, 95.4815673828125, 96.78953552246094, 99.28656005859375, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 99.7621841430664, 99.88109588623047, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[79.94164562225342, 65.28105235099792, 45.71472203731537, 27.194209456443787, 16.141066133975983, 8.899913668632507, 5.523029051721096, 4.454297497868538, 2.290982287377119, 1.2912689801305532, 0.5621603932231665, 0.33839644491672516, 0.20687389699742198, 0.15994632616639137, 0.12820824282243848, 0.11703529173973948, 0.093542012386024, 0.097585717914626, 0.5813554364722222, 0.4721522433683276, 0.33712789602577686, 0.12212724378332496, 0.06581782887224108, 0.05215055536245927, 0.04232009110273793, 0.03399667068151757, 0.03210525866597891, 0.02832847589161247, 0.025690697482787073, 0.02350049011874944]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 93.22 %, Loss: 6.5386 
